In [13]:
import requests as r
import re
import os
import logging
from datetime import datetime, timedelta
from dotenv import load_dotenv
from shapely import from_wkt
from shapely.geometry import Point, LineString, Polygon, MultiPolygon

In [4]:
load_dotenv()

True

In [5]:
def generic_request(url: str, headers: dict = {}, access_token: str = None, **kwargs) -> dict:

    if access_token:
        headers['Authorization'] = f"Bearer {access_token}"
    response = r.get(url, headers=headers, **kwargs)
    json_response = response.json()
    if response.status_code >= 400:
        raise Exception(json_response)

    return json_response

In [6]:
def get_access_token(client_id: str, client_secret: str) -> str:
    '''
    Supplies a temporary access token for of the OS NGD API
    Times out after 5 minutes
    Takes the project client_id and client_secret as input
    '''

    url = "https://api.os.uk/oauth2/token/v1"

    data = {
        "grant_type": "client_credentials"
    }

    response = r.post(
        url, 
        auth=(client_id, client_secret),
        data=data
    )

    json_response = response.json()
    if response.status_code >= 400:
        raise Exception(json_response)
    token = json_response["access_token"]

    return token

In [7]:
def OAauth2_manager(func: callable):

    def wrapper(*args, **kwargs):

        kwargs_ = kwargs.copy()

        try:
            access_token = os.environ.get('ACCESS_TOKEN')
            kwargs_['access_token'] = access_token
            return func(*args, **kwargs_)
        except Exception:
            client_id = os.environ.get('CLIENT_ID')
            client_secret = os.environ.get('CLIENT_SECRET')
            access_token = get_access_token(
                client_id=client_id,
                client_secret=client_secret
            )
            os.environ['ACCESS_TOKEN'] = access_token
            kwargs_['access_token'] = access_token
            return func(*args, **kwargs_)

    wrapper.__name__ = func.__name__ + '+OAuth2_manager'
    funcname = func.__name__
    wrapper.__doc__ = f"""
    An extension of the function {funcname} handling OAauth2 authorisation.
    IMPORTANT:
        CLIENT_ID and CLIENT_SECRET must be set as environment variables for this extension to work.
        This can be done using a .env file and load_dotenv()
    Docs for OAuth2 with Ordnance Survey data can be found at https://osdatahub.os.uk/docs/oauth2/overview
    The function automatically ensures a valid temporary access token (expiring after 5 minutes) is being used. This will either be an existing valid token, or a newly called one.

    ____________________________________________________
    Docs for {funcname}:
        {func.__doc__}
    """
    return wrapper

In [8]:
def wkt_to_spatial_filter(wkt, predicate='INTERSECTS'):
    '''Constructs a full spatial filter in conformance with the OGC API - Features standard from well-known-text (wkt)
    Currently, only 'Simple CQL' conformance is supported, therefore INTERSECTS is the only supported spatial predicate: https://portal.ogc.org/files/96288#rc_simple-cql'''
    return f'({predicate}(geometry,{wkt}))'

In [9]:
def construct_bbox_filter(
        bbox_tuple: tuple[float | int] | str = None,
        xmin: float | int = None,
        ymin: float | int = None,
        xmax: float | int = None,
        ymax: float | int = None
):
    if bbox_tuple:
        return str(bbox_tuple)[1:-1].replace(' ','')
    list_ = list()
    for z in [xmin, ymin, xmax, ymax]:
        if z == None:
            raise AttributeError('You must provide either bbox_tuple or all of [xmin, ymin, xmax, ymax]')
        list_.append(str(z))
    if xmin > xmax:
        raise ValueError('xmax must be greater than xmin')
    if ymin > ymax:
        raise ValueError('ymax must be greater than ymin')
    return ','.join(list_)

In [10]:
def construct_query_params(**params) -> str:
    '''
    Constructs a query string from a dictionary of key-value pairs.
    Refer to https://osdatahub.os.uk/docs/ofa/technicalSpecification for details about query parameters.
    The options are:
        - bbox
        - bbox-crs
        - crs
        - datetime
        - filter
        - filter-crs (can be supplied as a full http ref, or an integer)
        - filter-lang
        - limit
        - offset
    '''
    for p in ['crs', 'bbox-crs', 'filter-crs']:
        crs = params.get(p)
        if type(crs) == int:
            params[p] = f'http://www.opengis.net/def/crs/EPSG/0/{crs}'
    params_list = [f'{k}={v}' for k, v in params.items()]
    return '?' + '&'.join(params_list)

In [11]:
def construct_filter_param(**params):
    '''Constructs a set of key=value parameters into a filter string for an API query'''
    for k, v in params.items():
        if type(v) == str:
            params[k] = f"'{v}'"
    filter_list = [f"({k}={v})" for k, v in params.items()]
    return 'and'.join(filter_list)

In [12]:
def ngd_items_request(
    collection: str,
    query_params: dict = {},
    filter_params: dict = {},
    wkt = None,
    headers: dict = {},
    access_token: str = None,
    verbose: bool = False,
    **kwargs
) -> dict:
    """
    Calls items from the OS NGD API - Features
        - https://osdatahub.os.uk/docs/wfs/overview
        - https://docs.os.uk/osngd/accessing-os-ngd/access-the-os-ngd-api/os-ngd-api-features
    Parameters:
        collection (str) - the feature collection to call from. Feature collection names and details can be found at https://api.os.uk/features/ngd/ofa/v1/collections/
        query_params (dict, optional) - parameters to pass to the query as query parameters, supplied in a dictionary. Supported parameters are: bbox, bbox-crs, crs, datetime, filter, filter-crs, filter-lang, limit, offset
        filter_params (dict, optional) - OS NGD attribute filters to pass to the query within the 'filter' query_param. The can be used instead of or in addition to manually setting the filter in query_params.
            The key-value pairs will appended using the EQUAL TO [ = ] comparator. Any other CQL Operator comparisons must be set manually in query_params.
            Queryable attributes can be found in OS NGD codelists documentation https://docs.os.uk/osngd/code-lists/code-lists-overview, or by inserting the relevant collectionId into the https://api.os.uk/features/ngd/ofa/v1/collections/{{collectionId}}/queryables endpoint.
        wkt (string or shapely geometry object) - A means of searching a geometry for features. The search area(s) must be supplied in wkt, either in a string or as a Shapely geometry object.
            The function automatically composes the full INTERSECTS filter and adds it to the 'filter' query parameter.
            Make sure that 'filter-crs' is set to the appropriate value.
        headers (dict, optional) - Headers to pass to the query. These can include bearer-token authentication.
        access_token (str) - An access token, which will be added as bearer token to the headers.
        **kwargs: other generic parameters to be passed to the requests.get()

    Returns the features as a geojson, as per the OS NGD API.
    """

    query_params_ = query_params.copy()
    filter_params_ = filter_params.copy()
    headers_ = headers.copy()

    if filter_params_:
        filters = construct_filter_param(**filter_params_)
        current_filters = query_params_.get('filter')
        query_params_['filter'] = f'({current_filters})and{filters}' if current_filters else filters

    if wkt:
        spatial_filter = wkt_to_spatial_filter(wkt)
        current_filters = query_params_.get('filter')
        query_params_['filter'] = f'({current_filters})and{spatial_filter}' if current_filters else spatial_filter

    query_params_string = construct_query_params(**query_params_)
    url = f'https://api.os.uk/features/ngd/ofa/v1/collections/{collection}/items/{query_params_string}'
    if verbose:
        print(url)
    if access_token:
        headers_['Authorization'] = f"Bearer {access_token}"
    response = r.get(url, headers=headers_, **kwargs)
    json_response = response.json()

    if response.status_code >= 400:
        raise Exception(json_response)

    return json_response

In [13]:
def feature_limit_extension(func: callable):

    def wrapper(
        *args,
        request_limit: int = 50,
        feature_limit: int = None,
        query_params: dict = {},
        **kwargs
    ):

        query_params_ = query_params.copy()

        if 'offset' in query_params_:
            raise AttributeError('offset is not a valid argument for functions using this decorator.')

        items = list()

        batch_count, final_batchsize = divmod(feature_limit, 100) if feature_limit else (None, None)
        request_count = 0
        offset = 0

        if not(feature_limit) and not(request_limit):
            raise AttributeError('At least one of feature_limit or request_limit must be provided to prevent indefinitely numerous requests and high costs. However, there is no upper limit to these values.')

        while (request_count != request_limit) and (not(feature_limit) or offset < feature_limit):

            if request_count == batch_count:
                print('final batch of size', final_batchsize)
                query_params_['limit'] = final_batchsize
            query_params_['offset'] = offset

            json_response = func(*args, query_params=query_params_, **kwargs)
            request_count += 1
            items += json_response['features']

            if not [link for link in json_response['links'] if link['rel'] == 'next']:
                break
            
            offset += 100

        geojson = {
            "type": "FeatureCollection",
            "numberOfRequests": request_count,
            "totalNumberReturned": len(items),
            "timeStamp": datetime.now().isoformat(),
            "collection": kwargs.get('collection'),
            "source": "Compiled from code by Geovation from Ordnance Survey",
            "features": items
        }
        return geojson

    wrapper.__name__ = func.__name__ + '+feature_limit_extension'
    funcname = func.__name__
    wrapper.__doc__ = f"""
    This is an extension the {funcname} function, which returns OS NGD features. It serves to extend the maximum number of features returned above the default maximum 100 by looping through multiple requests.
    It takes the following arguments:
    - collection: The name of the collection to be queried.
    - request_limit: The maximum number of calls to be made to {funcname}. Default is 50.
    - feature_limit: The maximum number of features to be returned. Default is None.
    - query_params: A dictionary of query parameters to be passed to the function. Default is an empty dictionary.
    To prevent indefinite requests and high costs, at least one of feature_limit or request_limit must be provided, although there is no limit to the upper value these can be.
    It will make multiple requests to the function to compile all features from the specified collection, returning a dictionary with the features and metadata.

    ____________________________________________________
    Docs for {funcname}:
        {func.__doc__}
    """
    return wrapper

In [14]:
def multigeometry_search_extension(func: callable):

    def wrapper(*args, wkt: str, **kwargs):

        full_geom = from_wkt(wkt) if type(wkt) == str else wkt
        search_areas = list()

        is_single_geom = type(full_geom) in [Point, LineString, Polygon]
        partial_geoms = [full_geom] if is_single_geom else full_geom.geoms

        for search_area, geom in enumerate(partial_geoms):
            json_response = func(*args, wkt=geom, **kwargs)
            json_response['searchAreaNumber'] = search_area
            search_areas.append(json_response)

        geojson = {
            "type": "FeatureCollection",
            "searchAreas": search_areas
        }

        return geojson

    wrapper.__name__ = func.__name__ + '+multigeometry_search_extension'
    funcname = func.__name__
    wrapper.__doc__ = f"""
    An alternative means of returning OS NGD features for a search area which is a Multi-Geometry (MultiPoint, MultiLinestring, or MultiPolygon), which will in some cases improve speed, performance, and prevent the call from timing out.
    Extends to {funcname} function.
    Each component shape of the multi-geometry will be searched in turn using the {funcname} function.
    The results are returned in a quasi-GeoJSON format, with features returned under 'searchAreas' in a list, where each item is a json object of results from one search area.
    The search areas are labelled numerically, with the number stored under 'searchAreaNumber'.
    NOTE: If a limit is supplied for the maximum number of features to be returned or requests to be made, this will apply to each search area individually, not to the overall number of results.

    ____________________________________________________
    Docs for {funcname}:
        {func.__doc__}
    """
    return wrapper

In [15]:
def multiple_collections_extension(func: callable) -> dict:

    def wrapper(collections: list[str], *args, **kwargs):

        results = dict()
        for c in collections:
            json_response = func(c, *args, **kwargs)
            results[c] = json_response
        return results
    
    wrapper.__name__ = func.__name__ + '+multiple_collections_extension'
    funcname = func.__name__
    wrapper.__doc__ = f"""
    Extents the {funcname} function to handle multiple collections.
    Takes a list of collection names as input, alongside any other parameters which are passed to {funcname}.
    The function {funcname} will be run for each collection in turn, with the results returned in a dictionary mapping the collection names to the results.
    NOTE: If a limit is supplied for the maximum number of features to be returned or requests to be made, this will apply to each collection individually, not to the overall number of results.

    ____________________________________________________
    Docs for {funcname}:
        {func.__doc__}
    """
    return wrapper

In [16]:
def get_latest_collection_versions(flag_recent_updates: bool = True, recent_update_days: int = 31) -> tuple[dict[str: str], list[str]]:
    '''
    Returns the latest collection versions of each NGD collection.
    Feature collections follow the following naming convention: theme-collection-featuretype-version (eg. bld-fts-buildingline-2)
    The output of this function maps base feature collection names (theme-collection-featuretype) to the full name, including the latest version.
    This can be used to ensure that software is always using the latest version of a feature collection.
    More details on feature collection naming can be found at https://docs.os.uk/osngd/accessing-os-ngd/access-the-os-ngd-api/os-ngd-api-features/what-data-is-available
    '''

    response = r.get('https://api.os.uk/features/ngd/ofa/v1/collections/')
    collections_data = response.json()['collections']
    collections_list = [collection['id'] for collection in collections_data]
    collection_base_names = set([re.sub(r'-\d+$', '', c) for c in collections_list])
    output_lookup = dict()

    for base_name in collection_base_names:
        all_versions = [c for c in collections_list if c.startswith(base_name)]
        latest_version = max(all_versions, key=lambda c: int(c.split('-')[-1]))
        output_lookup[base_name] = latest_version

    recent_collections = None
    if flag_recent_updates:
        time_format = r'%Y-%m-%dT%H:%M:%SZ'
        recent_update_cutoff = datetime.now() - timedelta(days=recent_update_days)
        latest_versions_data = [c for c in collections_data if c['id'] in output_lookup.values()]
        recent_collections = list()
        for collection_data in latest_versions_data:
            version_startdate = collection_data['extent']['temporal']['interval'][0][0]
            time_obj = datetime.strptime(version_startdate, time_format)
            if time_obj > recent_update_cutoff:
                collection = collection_data['id']
                recent_collections.append(collection)
                logging.warning(f'{collection} is a recent version/update from the last {recent_update_days} days.')

    return output_lookup, recent_collections

In [17]:
def get_single_latest_collection(collection: str, **kwargs) -> str:
    '''
    Returns the latest collection of a given collection base.
    Input must be in the format theme-collection-featuretype (eg. bld-fts-buildingline)
    Output will complete the full name of the feature collection by appending the latest version number (eg. bld-fts-buildingline-2)
    More details on feature collection naming can be found at https://docs.os.uk/osngd/accessing-os-ngd/access-the-os-ngd-api/os-ngd-api-features/what-data-is-available
    '''
    latest_collections = get_latest_collection_versions(**kwargs)
    latest_collection = latest_collections[collection]
    return latest_collection

In [18]:
def comprehensive_enum_search(search_vals: str | list):

    if type(search_vals) == str:
        search_vals = [search_vals]
    search_vals = [sv.upper() for sv in search_vals]

    latest_versions = get_latest_collection_versions()
    latest_versions_list = list(latest_versions[0].values())

    output = dict()

    for collection in latest_versions_list:

        queryables = r.get(f'https://api.os.uk/features/ngd/ofa/v1/collections/{collection}/queryables').json()
        properties = queryables['properties']

        for attribute, data in properties.items():

            type_ = data['type'][0]
            if type_ == 'string':
                enum = data.get('enum')
                if not(enum):
                    continue
                matches = list()
                for sv in search_vals:
                    matches += [val for val in enum if sv in val.upper()]
                if matches:
                    output[collection] = output.get(collection, {})
                    output[collection][attribute] = matches

            elif type_ == 'array':
                items = data.get('items')
                if not(items):
                    continue
                enum = items.get('enum')
                if not(enum):
                    continue
                matches = list()
                for sv in search_vals:
                    matches += [val for val in enum if sv in val.upper()]
                if matches:
                    output[collection] = output.get(collection, {})
                    output[collection][attribute] = {'array':matches}

    return output

In [19]:
pilons = comprehensive_enum_search(['electricity','power','substation'])

In [20]:
pilons

{'trn-fts-roadtrackorpath-3': {'oslandusetierb': {'array': ['Electricity Distribution']}},
 'bld-fts-buildingpart-2': {'oslandusetierb': {'array': ['Electricity Distribution']}},
 'str-fts-structure-3': {'description': ['Electricity Metal Monopole',
   'Electricity Pylon'],
  'oslandusetierb': {'array': ['Electricity Distribution']}},
 'str-fts-structureline-1': {'description': ['Electricity Transmission Lines']},
 'lus-fts-site-2': {'description': ['Electricity Distribution Site',
   'Electricity Storage Site',
   'Electricity Sub Station',
   'Biogas Power Station',
   'Hydroelectric Power Generating',
   'Power Station',
   'Solar Power Site',
   'Tidal Power Station'],
  'oslandusetierb': {'array': ['Electricity Distribution']}},
 'wtr-fts-water-3': {'oslandusetierb': {'array': ['Electricity Distribution']}},
 'bld-fts-building-3': {'description': ['Electricity Distribution Facility',
   'Electricity Storage Facility',
   'Electricity Sub Station',
   'Biogas Power Station',
   'Hy

In [21]:
from shapely.geometry import LineString, GeometryCollection

In [22]:
x=Polygon({(400000,400000), (400050,400000), (400050,400050), (400000,40050)})
y=Polygon({(500000,500000), (500050,500000), (500050,500050), (500000,50050)})

In [14]:
l = LineString([(399990,399990),(399970,399970)])

In [17]:
type(l) == LineString

True

In [24]:
geomcollect = GeometryCollection([x,l])

In [25]:
z=MultiPolygon([x,y])

# Example Ways of Building Custom APIs by Combining Wrappers

#### All possible combos with OAuth2

In [27]:
items_auth = OAauth2_manager(ngd_items_request)
items_auth_limit = feature_limit_extension(items_auth)
items_auth_limit_geom = multigeometry_search_extension(items_auth_limit)
items_auth_limit_geom_col = multiple_collections_extension(items_auth_limit_geom)

items_auth_geom = multigeometry_search_extension(items_auth)
items_auth_geom_col = multiple_collections_extension(items_auth_geom)

items_auth_limit_col = multiple_collections_extension(items_auth_limit)

items_auth_col = multiple_collections_extension(items_auth)


In [28]:
get_latest_collection_versions()

({'trn-rami-specialdesignationpoint': 'trn-rami-specialdesignationpoint-1',
  'trn-ntwk-pavementlink': 'trn-ntwk-pavementlink-1',
  'trn-ntwk-connectingnode': 'trn-ntwk-connectingnode-1',
  'bld-fts-buildingline': 'bld-fts-buildingline-1',
  'gnm-fts-namedroadjunction': 'gnm-fts-namedroadjunction-1',
  'trn-ntwk-path': 'trn-ntwk-path-1',
  'lnd-fts-landform': 'lnd-fts-landform-1',
  'lnd-fts-landformline': 'lnd-fts-landformline-1',
  'lus-fts-siteaccesslocation': 'lus-fts-siteaccesslocation-1',
  'trn-ntwk-roadnode': 'trn-ntwk-roadnode-1',
  'trn-fts-cartographicraildetail': 'trn-fts-cartographicraildetail-1',
  'str-fts-compoundstructure': 'str-fts-compoundstructure-2',
  'trn-ntwk-pathlink': 'trn-ntwk-pathlink-1',
  'lnd-fts-landpoint': 'lnd-fts-landpoint-1',
  'trn-ntwk-pathnode': 'trn-ntwk-pathnode-1',
  'trn-fts-roadtrackorpath': 'trn-fts-roadtrackorpath-3',
  'trn-rami-restriction': 'trn-rami-restriction-1',
  'bld-fts-buildingpart': 'bld-fts-buildingpart-2',
  'str-fts-fieldboun

In [29]:
items_auth_limit(
    collection='lus-fts-site-2',
    query_params={
        'bbox':construct_bbox_filter((400000,400000,405000,405000)),
        'bbox-crs':27700,
        'filter':"oslandusetierb ACONTAINS ['Electricity Distribution']"
    },
    verbose=True
)

https://api.os.uk/features/ngd/ofa/v1/collections/lus-fts-site-2/items/?bbox=400000,400000,405000,405000&bbox-crs=http://www.opengis.net/def/crs/EPSG/0/27700&filter=oslandusetierb ACONTAINS ['Electricity Distribution']&offset=0
https://api.os.uk/features/ngd/ofa/v1/collections/lus-fts-site-2/items/?bbox=400000,400000,405000,405000&bbox-crs=http://www.opengis.net/def/crs/EPSG/0/27700&filter=oslandusetierb ACONTAINS ['Electricity Distribution']&offset=0


{'type': 'FeatureCollection',
 'numberOfRequests': 1,
 'totalNumberReturned': 2,
 'timeStamp': '2025-01-02T16:57:04.279219',
 'collection': 'lus-fts-site-2',
 'source': 'Compiled from code by Geovation from Ordnance Survey',
 'features': [{'id': 'c583336b-cf3a-4902-b33a-19b830815337',
   'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-1.9964562, 53.5311357],
       [-1.9964758, 53.5310871],
       [-1.9964155, 53.5310781],
       [-1.9963973, 53.5311267],
       [-1.9964562, 53.5311357]]]]},
   'properties': {'osid': 'c583336b-cf3a-4902-b33a-19b830815337',
    'toid': None,
    'theme': 'Land Use',
    'status': None,
    'nlud_code': 'U061',
    'changetype': 'New',
    'name1_text': None,
    'name2_text': None,
    'description': 'Electricity Sub Station',
    'matcheduprn': 10024991830,
    'stakeholder': None,
    'versiondate': '2024-09-06',
    'mainbuildingid': None,
    'name1_language': None,
    'name2_language': None,
    'oslandusetiera':

In [30]:
items_auth_limit(
    collection='bld-fts-building-3',
    query_params={
        'bbox':construct_bbox_filter((400000,400000,405000,405000)),
        'bbox-crs':27700,
        'filter':r"(description LIKE '%electricity%')OR(description LIKE '%power%')"
    },
    verbose=True
)

https://api.os.uk/features/ngd/ofa/v1/collections/bld-fts-building-3/items/?bbox=400000,400000,405000,405000&bbox-crs=http://www.opengis.net/def/crs/EPSG/0/27700&filter=(description LIKE '%electricity%')OR(description LIKE '%power%')&offset=0


{'type': 'FeatureCollection',
 'numberOfRequests': 1,
 'totalNumberReturned': 2,
 'timeStamp': '2025-01-02T16:57:04.743942',
 'collection': 'bld-fts-building-3',
 'source': 'Compiled from code by Geovation from Ordnance Survey',
 'features': [{'id': '15144597-2b91-426e-9fab-71d5a0bd6831',
   'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-1.9976073, 53.5326704],
      [-1.9976069, 53.5326724],
      [-1.9976013, 53.5327018],
      [-1.9975455, 53.5326955],
      [-1.9975527, 53.5326724],
      [-1.9975545, 53.5326668],
      [-1.9976073, 53.5326704]]]},
   'properties': {'osid': '15144597-2b91-426e-9fab-71d5a0bd6831',
    'theme': 'Buildings',
    'isinsite': True,
    'changetype': 'New',
    'buildinguse': 'Unknown',
    'description': 'Electricity Sub Station',
    'versiondate': '2024-07-20',
    'connectivity': 'Standalone',
    'primarysiteid': 'f31855a6-30f1-452d-ba13-35298b3c712f',
    'sitereference': [{'siteid': 'f31855a6-30f1-452d-ba13-35298b3c71

In [31]:
items_auth_limit(
    collection='wtr-ntwk-waterlink-1',
    query_params={
        'bbox':construct_bbox_filter((400000,400000,405000,405000)),
        'bbox-crs':27700
    },
    verbose=True
)

https://api.os.uk/features/ngd/ofa/v1/collections/wtr-ntwk-waterlink-1/items/?bbox=400000,400000,405000,405000&bbox-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=0
https://api.os.uk/features/ngd/ofa/v1/collections/wtr-ntwk-waterlink-1/items/?bbox=400000,400000,405000,405000&bbox-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=100
https://api.os.uk/features/ngd/ofa/v1/collections/wtr-ntwk-waterlink-1/items/?bbox=400000,400000,405000,405000&bbox-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=200
https://api.os.uk/features/ngd/ofa/v1/collections/wtr-ntwk-waterlink-1/items/?bbox=400000,400000,405000,405000&bbox-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=300
https://api.os.uk/features/ngd/ofa/v1/collections/wtr-ntwk-waterlink-1/items/?bbox=400000,400000,405000,405000&bbox-crs=http://www.opengis.net/def/crs/EPSG/0/27700&offset=400
https://api.os.uk/features/ngd/ofa/v1/collections/wtr-ntwk-waterlink-1/items/?bbox=400000,400000,405000,405000&bbox-crs=http://

{'type': 'FeatureCollection',
 'numberOfRequests': 9,
 'totalNumberReturned': 810,
 'timeStamp': '2025-01-02T16:57:10.632710',
 'collection': 'wtr-ntwk-waterlink-1',
 'source': 'Compiled from code by Geovation from Ordnance Survey',
 'features': [{'id': '00415101-a6d2-4caf-b7e6-6a5faa1ea097',
   'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[-1.9719947, 53.5001669],
     [-1.9720123, 53.5001418],
     [-1.9720758, 53.5000984]]},
   'properties': {'osid': '00415101-a6d2-4caf-b7e6-6a5faa1ea097',
    'toid': 'osgb5000005148131569',
    'theme': 'Water',
    'width': 7.8,
    'nameid': 'b2bb68d0-40d9-4baf-a6b4-f56568c760a6',
    'endnode': 'e5d8b181-8d04-4ef7-ad83-8af7c3e9e7ed',
    'primacy': 1,
    'gradient': 2.35,
    'startnode': '5d6c3b03-e6a0-475c-9c0d-8d1f56003ccd',
    'watertype': 'Inland',
    'changetype': 'New',
    'name1_text': 'Ogden Brook',
    'name2_text': None,
    'permanence': 'Permanent',
    'catchmentid': '7100',
    'description': 'W

## Demos

In [32]:
everything_test = items_auth_limit_geom_col(
    collections=['lnd-fts-land-3','wtr-fts-water-1','bld-fts-building-3'],
    query_params={'filter-crs':27700},
    request_limit=3,
    verbose=True
)

TypeError: multigeometry_search_extension.<locals>.wrapper() missing 1 required keyword-only argument: 'wkt'